In [3]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder
from cosapp.drivers import NonLinearSolver, RunOnce, MonteCarlo, RunSingleCase
from cosapp.utils.distributions import Normal
import sys
sys.path.append('./model')
from Earth import Earth
import numpy as np

earth = Earth("earth")

l = 10 #Rocket's length on the plot
dt = 0.05 #Time-step


from cosapp.utils.distributions import Normal
from cosapp.drivers import MonteCarlo
from cosapp.recorders import DataFrameRecorder

In [4]:

def run_analysis(syst, draws=10, linear=True):
    syst.drivers.clear()  # Remove all drivers on the System

    runonce = syst.add_driver(RunOnce("runonce"))
    driver = syst.add_driver(RungeKutta(order=4, dt=dt))
    driver.add_child(NonLinearSolver('solver', factor=1.0))
    driver.time_interval = (0, 40)

    # Define a simulation scenario
    driver.set_scenario(
        init = {
        'Traj.r' : np.array([0., 0., l/2]),
        'Rocket.Kin.v' : np.array([0,0,0]),
        'Rocket.Kin.ar' : np.array([0, -np.pi/2 + 0.2, 0]),
        'Rocket.Kin.av' : np.zeros(3),
        'Para.DynPar.r1' : np.array([0., 0., l/2]), #(should be l because the parachute is at the tip of the rocket)
        'Para.DynPar.r2' : np.array([0., 0., l/2]),
        'Para.DynPar.v1' : np.array([0,0,0]),
        'Para.DynPar.v2' : np.array([0,0,0]),
        'Traj.ParaDepStatus': False
    },
    stop='Para.DynPar.r1[2] < -1'
    )

    syst.run_drivers()
    print("MONTECARLO")
    # Montecarlo
    syst.drivers.clear()
    montecarlo = syst.add_driver(MonteCarlo('mc'))
    montecarlo.add_recorder(DataFrameRecorder(includes=['Rocket.Kin.ar', 'Traj.r']))
    montecarlo.add_child(driver)
    montecarlo.draws = draws
    # montecarlo.linear = linear

    # parameters for uncertainties in the data
    cant_attr = syst.Rocket.Aero.Coefs.inwards.get_details('delta')
    # Set the distribution around the current value
    cant_attr.distribution = Normal(best=.05, worst=-0.05)
    montecarlo.add_random_variable('Rocket.Aero.Coefs.delta')

    # Computation
    syst.run_drivers()

    return montecarlo.recorder.export_data()


In [5]:

results = run_analysis(earth, draws=10, linear=False)

___PARACHUTE DEPLOYMENT___


Point 1 X Coordinate at Deployment:  78.45898728473324 m


Point 1 Y Coordinate at Deployment:  -1.4159470513277265e-15 m


Point 1 Z Coordinate at Deployment:  231.9151213348103 m


Point 2 X Coordinate at Deployment:  78.24128125589692 m


Point 2 Y Coordinate at Deployment:  -1.4186611931462264e-15 m


Point 2 Z Coordinate at Deployment:  231.46500569965826 m


___PARACHUTE DEPLOYMENT___
Parachute fully deployed !
MONTECARLO


c:\Users\aoyag\Desktop\Etudes\PSC\CosappRepo\rocket-twin\UI\./model\Aerodynamics\Coefficients.py:194: RuntimeWarning: invalid value encountered in scalar power
  C_D_fin_pressure = (1 - M**2)**(-0.417) - 1


___PARACHUTE DEPLOYMENT___


Point 1 X Coordinate at Deployment:  82.54323118889045 m


Point 1 Y Coordinate at Deployment:  -1.4190633056694566e-15 m


Point 1 Z Coordinate at Deployment:  244.54926477153668 m


Point 2 X Coordinate at Deployment:  82.32610899882637 m


Point 2 Y Coordinate at Deployment:  -1.4218208989364267e-15 m


Point 2 Z Coordinate at Deployment:  244.09886721983176 m


___PARACHUTE DEPLOYMENT___
Parachute fully deployed !
___PARACHUTE DEPLOYMENT___


Point 1 X Coordinate at Deployment:  82.54323118889046 m


Point 1 Y Coordinate at Deployment:  -1.4190633056694566e-15 m


Point 1 Z Coordinate at Deployment:  244.54926477153668 m


Point 2 X Coordinate at Deployment:  82.32610899882638 m


Point 2 Y Coordinate at Deployment:  -1.4218208989364267e-15 m


Point 2 Z Coordinate at Deployment:  244.09886721983176 m


___PARACHUTE DEPLOYMENT___
Parachute fully deployed !
___PARACHUTE DEPLOYMENT___


Point 1 X Coordinate at Deployment:  82.54323118889046 m


Point 1 Y 

In [7]:

import plotly.graph_objs as go
from plotly.subplots import make_subplots

traj = np.asarray(results['Traj.r'].tolist())
print(traj[:, 0])

# Create the figure object
fig = make_subplots(rows=1, cols=1)
fig.layout.title = "Probability"
fig.layout.yaxis.title = 'Y Position'

fig.add_trace(
    go.Scatter(
        x=traj[:,0],
        y=traj[:,1],
        mode = 'markers'
    ),
    row = 1,
    col = 1,
)
fig.get_subplot(1, 1).xaxis.title = "X Position"

fig.show()

[-116.58053476  475.26448663  475.26448662  200.64817675  437.58289507
  200.64817674  437.58289507  247.77813642  400.62071504  378.48341897]
